## Imports

In [1]:
import pandas as pd

import surprise
from surprise.prediction_algorithms import *
import pandas as pd
import numpy as np
import datetime as dt

from surprise import Dataset
from surprise import Reader

from surprise.prediction_algorithms import knns
from surprise.similarities import cosine, msd, pearson
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from surprise.model_selection import cross_validate

from nltk.tokenize import RegexpTokenizer

import pickle 

## EDA

In [2]:
movies = pd.read_csv('../../ml-latest-small/movies.csv')

In [3]:
link = pd.read_csv('../../ml-latest-small/links.csv')

In [4]:
rating = pd.read_csv('../../ml-latest-small/ratings.csv')

In [5]:
tags = pd.read_csv('../../ml-latest-small/tags.csv')

In [6]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [7]:
movies.isna().sum()

movieId    0
title      0
genres     0
dtype: int64

In [8]:
movies.title.value_counts()

Confessions of a Dangerous Mind (2002)    2
Emma (1996)                               2
War of the Worlds (2005)                  2
Eros (2004)                               2
Saturn 3 (1980)                           2
                                         ..
Mindhunters (2004)                        1
Missing in Action (1984)                  1
Short Cuts (1993)                         1
Summer Wars (Samâ wôzu) (2009)            1
Fear of a Black Hat (1994)                1
Name: title, Length: 9737, dtype: int64

In [9]:
#Check to see if there are any duplicate titles
movies.title.duplicated().sum()

5

In [10]:
#Drop the 5 duplicated movie titles
movies.drop_duplicates(subset='title', inplace=True)

In [11]:
#Sanity check to ensure all duplicates were dropped from title column

movies.title.duplicated().sum()

0

In [12]:
movies[movies.title.duplicated() == True]

,movieId,title,genres


In [13]:
#Check length of DataFrame

len(movies)

9737

In [14]:
# Split title column into two new columns: Title and year 

movies['Title'] = movies['title'].str.split('(', n=1, expand=True)[0]

movies['year_released'] = movies['title'].str.split('(', n=1, expand=True)[1]

In [15]:
# Drop original column title

movies.drop(columns='title', inplace=True)

In [16]:
# Sanity Check 
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995)
1,2,Adventure|Children|Fantasy,Jumanji,1995)
2,3,Comedy|Romance,Grumpier Old Men,1995)
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995)
4,5,Comedy,Father of the Bride Part II,1995)
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017)
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017)
9739,193585,Drama,Flint,2017)
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018)


In [17]:
# Format year_released column

movies['year_released'] = movies.year_released.str.replace(')', '')

In [18]:
# Sanity Check to ensure formatting was completed
movies

,movieId,genres,Title,year_released
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995
1,2,Adventure|Children|Fantasy,Jumanji,1995
2,3,Comedy|Romance,Grumpier Old Men,1995
3,4,Comedy|Drama|Romance,Waiting to Exhale,1995
4,5,Comedy,Father of the Bride Part II,1995
...,...,...,...,...
9737,193581,Action|Animation|Comedy|Fantasy,Black Butler: Book of the Atlantic,2017
9738,193583,Animation|Comedy|Fantasy,No Game No Life: Zero,2017
9739,193585,Drama,Flint,2017
9740,193587,Action|Animation,Bungo Stray Dogs: Dead Apple,2018


In [19]:
link

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [20]:
rating

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [21]:
#converted timestamp to datetime
rating['timestamp'] = pd.to_datetime(rating['timestamp'], unit='s')

In [22]:
rating.rating.value_counts(normalize=True)

4.0    0.265957
3.0    0.198808
5.0    0.131015
3.5    0.130271
4.5    0.084801
2.0    0.074884
2.5    0.055040
1.0    0.027877
1.5    0.017762
0.5    0.013586
Name: rating, dtype: float64

In [23]:
rating.duplicated().sum()

0

In [24]:
rating.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [25]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [26]:
movie_rating = movies.merge(rating, on='movieId', how='outer')

In [27]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,Adventure|Animation|Children|Comedy|Fantasy,Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100849,64997,NaN,NaN,NaN,68.0,2.5,2008-12-28 20:55:15
100850,144606,NaN,NaN,NaN,111.0,4.0,2018-01-31 23:27:37
100851,147002,NaN,NaN,NaN,318.0,4.0,2017-08-08 15:45:52
100852,26958,NaN,NaN,NaN,509.0,3.5,2015-07-04 17:42:33


In [28]:
movie_rating.dropna(inplace=True)

In [29]:
movie_rating['genres']=[row.strip().lower().replace('|',',') for row in movie_rating['genres']]

In [30]:
movie_rating['userId'].nunique()

610

In [31]:
stats = movie_rating.filter(['rating', 'timestamp']).describe()
stats

,rating
count,100813.000000
mean,3.501557
std,1.042494
min,0.500000
25%,3.000000
50%,3.500000
75%,4.000000
max,5.000000


In [32]:
movie_rating

,movieId,genres,Title,year_released,userId,rating,timestamp
0,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,1.0,4.0,2000-07-30 18:45:03
1,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,5.0,4.0,1996-11-08 06:36:02
2,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,7.0,4.5,2005-01-25 06:52:26
3,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,15.0,2.5,2017-11-13 12:59:30
4,1,"adventure,animation,children,comedy,fantasy",Toy Story,1995,17.0,4.5,2011-05-18 05:28:03
...,...,...,...,...,...,...,...
100843,193581,"action,animation,comedy,fantasy",Black Butler: Book of the Atlantic,2017,184.0,4.0,2018-09-16 14:44:42
100844,193583,"animation,comedy,fantasy",No Game No Life: Zero,2017,184.0,3.5,2018-09-16 14:52:25
100845,193585,drama,Flint,2017,184.0,3.5,2018-09-16 14:56:45
100846,193587,"action,animation",Bungo Stray Dogs: Dead Apple,2018,184.0,3.5,2018-09-16 15:00:21


In [33]:
# movie_rating['genres'] = movie_rating['genres'].to_list()

In [34]:
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)
movie_rating['genres'] = [tokenizer.tokenize(token) for token in movie_rating['genres']]

## Split

In [46]:
train, test = train_test_split(movie_rating, random_state=42)

In [47]:
reader = Reader(rating_scale=(1, 5))
train_data = Dataset.load_from_df(train[['userId', 'movieId', 'rating']], reader)

In [48]:
test_data = Dataset.load_from_df(test[['userId', 'movieId', 'rating']], reader)

## Tuning

### KNNBasics

In [57]:
param_grid = {'k':[10, 50, 100],'min_k': [1, 5, 10]}
base_model = GridSearchCV(KNNBasic,param_grid=param_grid,joblib_verbose=5)
base_model.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.1s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.4s remaining:    0.0s


...

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.5s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.


[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.6s remaining:    0.0s


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computi

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.0min finished


In [58]:
base_model.best_params

{'rmse': {'k': 10, 'min_k': 5}, 'mae': {'k': 10, 'min_k': 5}}

In [59]:
base_model.best_estimator

{'rmse': <surprise.prediction_algorithms.knns.KNNBasic at 0x21b06e693a0>,
 'mae': <surprise.prediction_algorithms.knns.KNNBasic at 0x21b06f53d00>}

In [41]:
# param_grid = {'n_factors':[20, 100],'n_epochs': [5, 10], 'lr_all': [0.002, 0.005],
#               'reg_all': [0.4, 0.6]}
# gs_model = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
# gs_model.fit(jokes)

## Cross Validation

In [50]:
#instantiate KNN model 
model1=KNNBasic(k=10, min_k=5)


In [54]:
#instaniatite and perform cross val with training data
cross_validate(model1, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNBasic on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9518  0.9618  0.9554  0.9554  0.9475  0.9544  0.0047  
MAE (testset)     0.7288  0.7382  0.7367  0.7311  0.7255  0.7321  0.0048  
Fit time          0.18    0.20    0.16    0.16    0.19    0.18    0.02    
Test time         1.01    0.98    0.91    1.01    0.99    0.98    0.04    


{'test_rmse': array([0.95181613, 0.96176338, 0.95543695, 0.9553569 , 0.94753704]),
 'test_mae': array([0.72880515, 0.73819572, 0.73669445, 0.73110102, 0.72550763]),
 'fit_time': (0.18295812606811523,
  0.1977369785308838,
  0.16095972061157227,
  0.15795373916625977,
  0.19150471687316895),
 'test_time': (1.0143232345581055,
  0.9763960838317871,
  0.9075298309326172,
  1.0062344074249268,
  0.9882290363311768)}

In [55]:
train_data

In [56]:
#use model to predict
model1.predict(uid=10, iid = 20)
#this model just predicted what user 10 would predict movie 20

Prediction(uid=10, iid=20, r_ui=None, est=2.70452477452635, details={'actual_k': 7, 'was_impossible': False})

looking at the output above we can determine that our model predicted that use 10 would rate movie 20 a 2.5 give or take our RSME which is .95.

## Models

### SVD

In [60]:
model2 = SVD()

In [61]:
model2

In [63]:
cross_validate(model2, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8848  0.8780  0.8811  0.8836  0.8737  0.8803  0.0040  
MAE (testset)     0.6826  0.6759  0.6791  0.6834  0.6722  0.6786  0.0042  
Fit time          4.28    4.54    4.25    4.13    4.11    4.26    0.15    
Test time         0.14    0.15    0.21    0.13    0.14    0.16    0.03    


{'test_rmse': array([0.88484386, 0.87798873, 0.88112471, 0.88356574, 0.87373233]),
 'test_mae': array([0.68264667, 0.67589524, 0.67908356, 0.68336206, 0.67220202]),
 'fit_time': (4.283322095870972,
  4.541794300079346,
  4.246084928512573,
  4.132572650909424,
  4.1067891120910645),
 'test_time': (0.1435847282409668,
  0.1495969295501709,
  0.2124614715576172,
  0.133162260055542,
  0.1415238380432129)}

In [64]:
param_grid = {'n_factors':[10,20,50],'n_epochs': [5, 10, 15], 'lr_all': [0.002,0.005,0.01],
             'reg_all': [0.2,0.4,0.6]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    1.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    1.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    2.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed:  8.4min finished


In [65]:
gs_model2.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2},
 'mae': {'n_factors': 10, 'n_epochs': 15, 'lr_all': 0.01, 'reg_all': 0.2}}

In [66]:
cross_validate(SVD(n_factors=10,n_epochs=15,lr_all=0.01,reg_all=0.2), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8708  0.8737  0.8742  0.8823  0.8722  0.8747  0.0040  
MAE (testset)     0.6740  0.6741  0.6724  0.6787  0.6738  0.6746  0.0021  
Fit time          1.27    1.28    1.18    1.13    1.11    1.20    0.07    
Test time         0.23    0.15    0.11    0.20    0.13    0.16    0.04    


{'test_rmse': array([0.87082381, 0.87374972, 0.87422215, 0.8823252 , 0.87216949]),
 'test_mae': array([0.6739614 , 0.6740719 , 0.67237973, 0.67867877, 0.67376185]),
 'fit_time': (1.2701070308685303,
  1.2810485363006592,
  1.1840288639068604,
  1.1281569004058838,
  1.1121435165405273),
 'test_time': (0.2282123565673828,
  0.14960908889770508,
  0.11420488357543945,
  0.20386815071105957,
  0.12566351890563965)}

In [67]:
param_grid = {'n_factors':[5,10,15],'n_epochs': [15, 30, 45], 'lr_all': [0.01,0.05,0.1],
             'reg_all': [0.1,0.2,0.3]}
gs_model2 = GridSearchCV(SVD,param_grid=param_grid,joblib_verbose=5)
gs_model2.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.9s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 405 out of 405 | elapsed: 15.9min finished


In [68]:
gs_model2.best_params

{'rmse': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1},
 'mae': {'n_factors': 15, 'n_epochs': 45, 'lr_all': 0.01, 'reg_all': 0.1}}

In [69]:
cross_validate(SVD(n_factors=5,n_epochs=15,lr_all=0.01,reg_all=0.1), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8761  0.8705  0.8756  0.8703  0.8679  0.8721  0.0032  
MAE (testset)     0.6756  0.6723  0.6745  0.6697  0.6688  0.6722  0.0027  
Fit time          1.07    1.01    1.09    1.05    1.04    1.05    0.03    
Test time         0.12    0.21    0.13    0.12    0.12    0.14    0.03    


{'test_rmse': array([0.87614748, 0.8704698 , 0.87562339, 0.87026335, 0.86788248]),
 'test_mae': array([0.67559445, 0.67230334, 0.67452303, 0.66967483, 0.66875356]),
 'fit_time': (1.0683891773223877,
  1.010462999343872,
  1.0913703441619873,
  1.0482027530670166,
  1.0409338474273682),
 'test_time': (0.12315225601196289,
  0.2079181671142578,
  0.12566685676574707,
  0.11771726608276367,
  0.12195634841918945)}

### NMF

In [70]:
model3 = NMF()

In [71]:
cross_validate(model3, train_data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9328  0.9377  0.9423  0.9370  0.9434  0.9387  0.0038  
MAE (testset)     0.7147  0.7240  0.7264  0.7192  0.7213  0.7211  0.0041  
Fit time          5.81    5.53    5.27    5.40    5.17    5.44    0.23    
Test time         0.11    0.10    0.10    0.11    0.10    0.10    0.01    


{'test_rmse': array([0.93284816, 0.93768521, 0.94233844, 0.93698465, 0.94344445]),
 'test_mae': array([0.71465602, 0.72401626, 0.72642912, 0.71919712, 0.72126479]),
 'fit_time': (5.813294172286987,
  5.531378984451294,
  5.267866849899292,
  5.399816513061523,
  5.1651341915130615),
 'test_time': (0.10914468765258789,
  0.09873628616333008,
  0.10291910171508789,
  0.1147153377532959,
  0.09774017333984375)}

In [72]:
param_grid = {'n_factors':[10,15,20],'n_epochs': [25, 50, 75]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    5.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    7.6s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:   10.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  4.2min finished


In [73]:
gs_model3.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 50},
 'mae': {'n_factors': 20, 'n_epochs': 50}}

In [74]:
cross_validate(NMF(n_factors=10,n_epochs=25), train_data, measures=['RMSE', 'MAE'], 
               cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.9737  0.9576  0.9742  0.9680  0.9654  0.9678  0.0061  
MAE (testset)     0.7332  0.7282  0.7381  0.7372  0.7299  0.7333  0.0039  
Fit time          2.36    2.38    2.46    2.28    2.40    2.37    0.06    
Test time         0.11    0.10    0.12    0.10    0.21    0.12    0.04    


{'test_rmse': array([0.97365973, 0.95758671, 0.9742184 , 0.96795361, 0.96540134]),
 'test_mae': array([0.73320433, 0.72818225, 0.73810892, 0.7372249 , 0.72988962]),
 'fit_time': (2.358416795730591,
  2.3750765323638916,
  2.4647014141082764,
  2.2810781002044678,
  2.3950228691101074),
 'test_time': (0.10604596138000488,
  0.09873604774475098,
  0.11570549011230469,
  0.09876561164855957,
  0.20545125007629395)}

In [75]:
param_grid = {'n_factors':[3,5,10],'n_epochs': [15, 20, 25]}
gs_model3 = GridSearchCV(NMF,param_grid=param_grid,joblib_verbose=5)
gs_model3.fit(train_data)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    4.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    5.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:  1.3min finished


In [76]:
gs_model3.best_params

{'rmse': {'n_factors': 10, 'n_epochs': 25},
 'mae': {'n_factors': 10, 'n_epochs': 25}}

## Deployment 

In [77]:
train['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(train.userId[10], x).est)
train.sort_values(by='est_rating_user10', ascending=False, inplace=True)

In [78]:
train

,movieId,genres,Title,year_released,userId,rating,timestamp,est_rating_user10
24110,1136,"[adventure, comedy, fantasy]",Monty Python and the Holy Grail,1975,445.0,5.0,2016-02-04 21:41:28,5.0
14263,527,"[drama, war]",Schindler's List,1993,444.0,5.0,1996-08-06 06:10:47,5.0
14324,527,"[drama, war]",Schindler's List,1993,608.0,4.0,2005-05-30 01:06:01,5.0
49465,2921,[western],High Plains Drifter,1973,57.0,5.0,2000-08-09 05:23:11,5.0
24074,1136,"[adventure, comedy, fantasy]",Monty Python and the Holy Grail,1975,292.0,4.0,2008-08-25 00:44:53,5.0
...,...,...,...,...,...,...,...,...
100828,189547,"[action, sci, fi]",Iron Soldier,2010,210.0,1.0,2018-06-08 19:26:51,1.0
95958,104017,"[action, adventure, sci, fi]",3 dev adam,Three Giant Men (1973,599.0,0.5,2018-02-23 06:59:05,1.0
49105,2892,"[action, drama]",New Rose Hotel,1998,603.0,1.0,2000-07-09 21:26:11,1.0
63502,4794,"[crime, horror, mystery]",Opera,1987,594.0,0.5,2005-02-21 01:52:28,1.0


In [79]:
user = input('userId: ')
genre = input('What genres are you interested in? ')
num_recs = input('How many recomendations would you like? ')

userId: 10
What genres are you interested in? drama
How many recomendations would you like? 10


In [80]:
genre_df = train[train['genres'].str.contains(genre)]

KeyError: "None of [Float64Index([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,\n              ...\n              nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],\n             dtype='float64', length=75609)] are in the [columns]"

In [ ]:
genre_df

In [ ]:
movies['est_rating_user10'] = train['movieId'].apply(lambda x: model3.predict(uid=user, iid=x).est)
movies.sort_values(by='est_rating_user10', ascending=False, inplace=True) 

In [ ]:
# for x in train['genre']:
#     if genre in x:
#         print(x)

In [ ]:
# if genre in train['genres']:
#     #print(genre)
#     print(train['genres'])
    

In [ ]:
train #['Title'].head(int(num_recs))

In [ ]:
movies #['Title'].head(int(num_recs))

In [ ]:
movies[]